In [11]:
import networkx as nx
import numpy as np

In [12]:
def generate_example_graph():
    """Generates example graph from lecture with soft predictions.
    """
    graph = nx.Graph()
    graph.add_nodes_from([
    (1, {'label': 0, 'p_s': 0.05}),
    (2, {"label": 0, 'p_s': 0.30}),
    (3, { 'p_s': 0.60}),
    (4, { 'p_s': 0.20}),
    (5, {'p_s': 0.90}),
    (6,{ "label": 1,'p_s': 0.60}),
    (7, {"label": 1, 'p_s': 0.95}),
    (8, { 'p_s': 0.4}),
    (9, { 'p_s': 0.80})])

    graph.add_edges_from([(1,2),(1,3), (2,3), (1,4), (3,4), (4,5), (4,6), (5,6), (5, 7), (6,8), (7,8), (7,9)])
    return graph

In [13]:
def compute_node_errors(graph : nx.Graph) -> nx.Graph:
    """Compute per-node prediction errors of base predictor.

    Args:
        grpah (nx.Graph): Input network with precomputed softlabels on nodes.

    Returns:
        nx.Graph: Input network with computed errors as node features.
    """
    for g in graph.nodes.data():
        # Check if node has "hard-label".
        if 'label' in g[1]:
            # Attention: 'label' == 1 corresponds to vector (1,0).
            g[1]['error'] = ( (g[1]['label']-g[1]['p_s']), (1-g[1]['label']-(1-g[1]['p_s'])) )
        # Otherwise error is zero anyways.
        else:
            g[1]['error'] = (0.0,0.0)
    
    return graph

In [14]:
def get_normalized_adj(graph : nx.Graph) -> np.array:
    """Compute normalized adjacency matrix of given networkx graph.

    Args:
        graph (nx.Graph): Input network.

    Returns:
        np.array: Normalized adj matrix.
    """
    adj = nx.adjacency_matrix(graph).todense()
    deg = graph.degree
    deg_list = []
    for d in deg:
        deg_list.append(1/d[1])

    deg = np.diag(deg_list)
    de12= np.sqrt(deg)

    A = np.dot(de12, np.dot(adj, de12))
    return A

In [15]:
def get_start_error_matrix(graph : nx.Graph) -> np.array:
    """Transform node labels storing node errors into numpy matrix.

    Args:
        graph (nx.Graph): Input network with nodewise prediction errors.

    Returns:
        np.array: Matrix storing nodes in rows and errors in columns.
    """
    error_list = []
    for g in graph.nodes.data():
        row = [g[1]['error'][0], g[1]['error'][1]]
        error_list.append(row)
    error_matrix = np.array(error_list)
    return error_matrix 

In [16]:
def correct(graph : nx.Graph, start_errors : np.array, adj_matrix : np.array, alpha : float, s : float) -> np.array:
    """Correct step of C&S method.

    Args:
        start_erros (np.array): Initial prediction errors. Rows correspond to nodes.
        adj_matrix (np.array): Normalized adjacency matrix of input network.
        alpha (float): Hyperparameter.

    Returns:
        np.array: Corrected soft label matrix.
    """
    NUM_RUNS = 100
    error_t = start_errors
    # Propagate error over network.
    for _ in range(NUM_RUNS):
        error_t = (1-alpha) * start_errors + alpha * (adj_matrix @ error_t)
    
    # Add diffused errors to predicted soft labels.
    soft_label_list = []
    for node in graph.nodes.data():
        soft_label = [node[1]['p_s'], 1-node[1]['p_s']]
        soft_label_list.append(soft_label)
    soft_label_matrix = np.array(soft_label_list)
    output = soft_label_matrix + s * error_t
    return output

In [17]:
def replace_corrected_labels(graph : nx.Graph, corrected_soft_labels : np.array) -> np.array:
    """Replaces corrected soft labels matrix with 0-1 labels for labeled nodes.

    Args:
        graph (nx.Graph): Input network.
        corrected_soft_labels (np.array): Corrected soft label matrix.

    Returns:
        np.array: Corrected soft label matrix with replaced 0-1 rows for labeled nodes.
    """
    for node in graph.nodes.data():
        node_id = node[0]
        # Check if node has "hard label".
        if 'label' in node[1]:
            # Change row to hard label.
            row = [node[1]['label'], 1-node[1]['label']]
            corrected_soft_labels[node_id-1, :] = row
    return corrected_soft_labels
        

In [18]:
def smooth(z_matrix : np.array, adj_matrix : np.array, alpha : float)-> np.array:
    """Run smoothing of errors.

    Args:
        z_matrix (np.array): Corrected and replaced soft labels with nodes as rows.
        adj_matrix (np.array): Normalized adjacency matrix of input network.
        alpha (float): Diffusion parameter for propagation.

    Returns:
        np.array: Resulting classification probabilities.
    """
    NUM_RUNS = 100
    z_t = z_matrix
    for _ in range(NUM_RUNS):
        z_t = (1-alpha) * z_matrix + alpha * (adj_matrix @ z_t)
    return z_t

In [19]:
def classify_nodes(graph : nx.Graph, results : np.array) -> dict:
    """Classify nodes based on compute probabilities of C&S.

    Args:
        graph (nx.Graph): Input network.
        results (np.array): Matrix storing class probabilities.

    Returns:
        dict: Each index stores class of corresponding node.
    """
    classes = dict()
    for node in graph.nodes.data():
        node_id = node[0]
        if 'label' in node[1]:
            node_class = node[1]['label']
        else:
            node_class = 1-np.argmax(results[node_id-1, :])
        classes[node_id] = node_class
    return classes    

In [20]:
graph = generate_example_graph()

# Compute node-wise prediction errors.
graph = compute_node_errors(graph)

# Get normalized adjcaceny matrix of graph.
normalized_adj = get_normalized_adj(graph)
start_errors = get_start_error_matrix(graph)

### CORRECT STEP ###
alpha = 0.85
s = 2
corrected_soft_labels = correct(graph, start_errors, normalized_adj, alpha, s)

# Replace soft labels for labeled nodes.
z_matrix = replace_corrected_labels(graph, corrected_soft_labels)

### SMOOTH ###
result_probs = smooth(z_matrix, normalized_adj, alpha)
 
# Classify nodes.
classification = classify_nodes(graph, result_probs)
print(classification)


{1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
